In [28]:
import numpy as np
from plotly import express as px
import copy
#import jax.numpy as jnp
import pandas as pd

from grape import controlField as cf
from grape import optimalControl as COPT
from grape import rf
from grape import Analysis
from grape import Bruker
from grape import config
from grape import propagation


# Set-up global parameters

In [29]:
# Define global parameters
global_parameters = config.Global(
    application = 'selective_excitation',
    N           = 256,
    T_s         = 5e-3,
    nucleus     = 'H',
    refoc_f     = 0.5, # 0.5232,
)

# Set-up control fields parameters (w1x, w1y, gz)

In [30]:
## Set control fields
# w1x in rad/s
TBP=15 #Possible values 5,10,15
pulse_slr_matlab=pd.read_csv('wx_slr_matlab_tbp'+str(TBP)+'.csv',delimiter=',',header=None)
w1x_initial_temp_value =pulse_slr_matlab.to_numpy()[0]

param_w1x = {'name':'wavelet','wavelet_nbLevels':5, 'wavelet_type':'db16'} #To use temporal discretization change 'wavelet' to 'temp'
w1x       = cf.ControlField(global_parameters=global_parameters, initial_temp_value=w1x_initial_temp_value, optimize=True, parametrization=param_w1x)

# w1y in rad/s
w1y_initial_temp_value = np.zeros(global_parameters.N) 

param_w1y = {'name':'temp', 'wavelet_nbLevels':5, 'wavelet_type':'db16'}
w1y       = cf.ControlField(global_parameters=global_parameters, initial_temp_value=w1y_initial_temp_value, optimize=False,parametrization=param_w1y)

# gz in T/m
gamma                 = 267.513e6 # 'H'
sliceThickness_mm     = 1 
g_init                = 2*np.pi*TBP/(gamma * 1e-3*(sliceThickness_mm)*global_parameters.T_s)
gz_initial_temp_value = g_init*np.ones(global_parameters.N) # in T/m

param_gz = {'name':'temp'}
gz       = cf.ControlField(global_parameters=global_parameters, initial_temp_value=gz_initial_temp_value,optimize=False,parametrization=param_gz)

# Define control fields dictionary
initial_control_fields = {'w1x':w1x, 'w1y':w1y, 'gz':gz}

trW_mm={15:0.1,10:0.1,5:0.2}

# Set-up isochromat properties

In [31]:
# Define different isochromat populations
population_1 = config.Population(
    application=global_parameters.application,
    T2=2e-3
)

# Generate global population list
population_list = []
population_list.append(population_1)

# Set up the problem geometry
geometry_parameters = config.Geometry(
    application         = global_parameters.application,
    transitionWidth_mm  = trW_mm[TBP], 
    max_stopBand_mm     = 5,
    nb_in_slice         = 41,
    nb_stopBand         = 200,
    sliceThickness_mm   = sliceThickness_mm,
    halfPos             = True #True use the isocromats in [0,5], False use: [-5,5]
)

# Set-up reference pulse

In [32]:
isoList_ref=propagation.generate_isoList(population_list, geometry_parameters, global_parameters)
isoList_ref=propagation.propagate_par(initial_control_fields,isoList_ref,global_parameters)
df=Analysis.plot_final_magn(isoList_ref)

# Set-up optimization parameters

In [33]:
# Set up the cost parameters
cost_parameters = config.Cost(
    application = global_parameters.application,
    cost_function='min_max_amplitude',#'min_max_amplitude',#'min_energy',
)
sol_folder='paper_solutions_TBP_'+str(TBP) #Folder to be used to save the solutions.

# Set up the algorithm & output parameters
algo_parameters = config.Algo(
    gradient_tolerance=1e-6,
    max_iter=2000,
    tolerance =1e-6,
    method='IPOPT',#'trust-constr'#'SLSQP'#'trust-constr'#,
    sol_folder=sol_folder
)

# Constraints
constraints = {
    'Mxy_inSlice_lb': {
         'isoList_comp':isoList_ref
       },  
    'Mxy_outSlice_ub':{
         'bound':0.02#*np.ones(geometry_parameters.nb_stopBand//2)              
    },
}

# Regroup all parameters
optim_parameters = {}
optim_parameters['global']      = global_parameters
optim_parameters['cost']        = cost_parameters
optim_parameters['geometry']    = geometry_parameters
optim_parameters['algo']        = algo_parameters
optim_parameters['constraints'] = constraints

# Run Optimization

In [34]:
oc = COPT.OC(optim_parameters=optim_parameters, population_list=population_list, control_fields=initial_control_fields)
optimized_control_fields = oc.run_optimization(control_fields=initial_control_fields)


t, rf_oc = optimized_control_fields['w1x'].plot_tempControl(plot=False)
df_oc= Analysis.plot_final_magn(isoList=oc.isoList,plot=False)
fig_m = px.line(x=df_oc['Position (mm)'], y=df_oc['Transverse magn.'])
fig_m.show()

Mxy_inSlice_lb: bounds are set by isoList_comp, any other bounds will be overwritten.
max amplitude = 17.89399 (uT)
max amplitude = 17.89399 (uT)
max amplitude = 16.67602 (uT)
max amplitude = 15.82707 (uT)
max amplitude = 14.57086 (uT)
max amplitude = 14.25502 (uT)
max amplitude = 14.27414 (uT)
max amplitude = 14.98823 (uT)
max amplitude = 14.90916 (uT)
max amplitude = 14.64020 (uT)
max amplitude = 14.80180 (uT)
max amplitude = 14.52321 (uT)
max amplitude = 14.53021 (uT)
max amplitude = 14.41006 (uT)
max amplitude = 14.37470 (uT)
max amplitude = 14.39413 (uT)
max amplitude = 14.41360 (uT)
max amplitude = 14.45374 (uT)
max amplitude = 14.41399 (uT)
max amplitude = 14.39126 (uT)
max amplitude = 14.38534 (uT)
max amplitude = 14.38136 (uT)
max amplitude = 14.38075 (uT)
max amplitude = 14.38143 (uT)
max amplitude = 14.38098 (uT)
max amplitude = 14.38095 (uT)
max amplitude = 14.38094 (uT)
max amplitude = 14.38094 (uT)
max amplitude = 14.38094 (uT)
max amplitude = 14.38094 (uT)


# Analysis

In [35]:
analysis_parameters = copy.deepcopy(optim_parameters)

analysis_parameters['geometry'].halfPos             = False
analysis_parameters['geometry'].max_stopBand_mm      = 10
analysis_parameters['geometry'].nb_stopBand          = 500
analysis_parameters['geometry'].nb_transBand        = 50

oc_analysis     = COPT.OC(optim_parameters=analysis_parameters, population_list=population_list, control_fields=optimized_control_fields)
oc_analysis_slr = COPT.OC(optim_parameters=analysis_parameters, population_list=population_list, control_fields=initial_control_fields)



oc_analysis.export_tempcontrol_uT(optimized_control_fields, sol_folder)
oc_analysis_slr.export_tempcontrol_uT(initial_control_fields, sol_folder+'/SLR')

oc_analysis.export_final_transverseMagn(sol_folder)
oc_analysis_slr.export_final_transverseMagn(sol_folder+'/SLR')

t, rf_oc = optimized_control_fields['w1x'].plot_tempControl(plot=False)
df_analysis = Analysis.plot_final_magn(oc_analysis.isoList,plot=False,export=True,folder=sol_folder)


t, rf_slr = initial_control_fields['w1x'].plot_tempControl(plot=False)
df_slr = Analysis.plot_final_magn(oc_analysis_slr.isoList,plot=False,export=True,folder=sol_folder+'/SLR')


fig_rf = px.line(x=t, y=[rf_oc, rf_slr], labels={'x':"Time (ms)", 'y':"(µt)"},title='Controls w_x')
fig_m = px.line(x=df_analysis['Position (mm)'], y=[df_analysis['Transverse magn.'], df_slr['Transverse magn.']],title='Tranverse Magenetization')
fig_p = px.line(x=df_analysis['Position (mm)'], y=[df_analysis['Phase (rad)'], df_slr['Phase (rad)']],title='Phase')

fig_rf.show()
fig_m.show()
fig_p.show()


# Finding the best refocusing setting

In [36]:
pos2 = df_analysis.loc[df_analysis['slice_status']=='in']['Position (mm)']
phase2 = df_analysis.loc[df_analysis['slice_status']=='in']['Phase (rad)']
p2=np.polyfit(pos2,phase2,1)
fig_p = px.line(x=pos2, y=[phase2, p2[0]*np.array(pos2)+p2[1]])
fig_p.show()

phf_s_oc=-p2[0]/(gamma*g_init*1e-3)
print('phase factor oc', phf_s_oc/global_parameters.T_s)


pos = df_slr.loc[df_slr['slice_status']=='in']['Position (mm)']
phase = df_slr.loc[df_slr['slice_status']=='in']['Phase (rad)']

#print(phase)
p=np.polyfit(pos,phase,1)
fig_p = px.line(x=pos, y=[phase, p[0]*np.array(pos)+p[1]])
fig_p.show()

phf_s=-p[0]/(gamma*g_init*1e-3)
print('phase factor slr', phf_s/global_parameters.T_s)

name='pulse_min_peak_T2_inf_signal_in_TBP_'+str(TBP)+'_refoc_f_'+str(round(phf_s_oc/global_parameters.T_s,3))+'_OC'
Bruker.exportBruker(optimized_control_fields,TBP,round(phf_s_oc/global_parameters.T_s,3),gamma,'exc',sol_folder,name)
Analysis.disp_rfAnalysis(optimized_control_fields,global_parameters,sol_folder,name)



phase factor oc 0.07757832041878127


phase factor slr 0.09505430259018147
